In [1]:
#pragma cling add_include_path("../../libtorch/include")
#pragma cling add_include_path("../../libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("../../libtorch/lib")
#pragma cling load("libtorch")

In [2]:
#include <iostream>
#include <cstddef>
#include <torch/torch.h>
#include <tuple>
#include <c10/util/Optional.h>

## step 1: create mock data x and y

In [3]:
std::pair<torch::Tensor, torch::Tensor> synthetic_data(torch::Tensor true_w, float true_b, int64_t num_samples) {

    auto X = torch::normal(0.0, 1.0, {num_samples, true_w.size(0)});
    auto y = torch::matmul(X, true_w) + true_b;
    y += torch::normal(0.0, 0.01, y.sizes());
    y = torch::reshape(y, {-1, 1});

    //return torch::cat({X, y}, 1);
    return {X, y};
 }

In [4]:
auto options = torch::TensorOptions().dtype(torch::kFloat).device(torch::kCPU);

// Generating the Dataset
torch::Tensor true_w = torch::tensor({2.0, -3.4}, options);
std::cout << true_w.size(0) << std::endl;
float true_b = 4.2;
int64_t num_samples = 20;

std::pair<torch::Tensor, torch::Tensor> data_and_label = synthetic_data(true_w, true_b, num_samples);

2


In [5]:
torch::Tensor X = data_and_label.first;
std::cout << X << std::endl;

torch::Tensor y = data_and_label.second;
std::cout << X << std::endl;

 1.5724 -2.3880
 0.7826 -2.2035
-0.5379 -0.4596
-1.3799  0.0037
-0.3815  1.4219
 0.2192  0.2297
 1.0010  0.5570
 0.6981 -0.8828
 0.7232 -1.2942
 0.8049 -0.5391
 0.3457 -1.5950
-2.1742  2.4071
 0.9592  1.3948
-0.8991 -0.3394
-1.2080 -1.3720
-0.8710 -0.3277
 0.3054 -0.9281
-0.4665 -0.8242
-0.5735 -1.1918
-1.2887 -1.7625
[ CPUFloatType{20,2} ]
 1.5724 -2.3880
 0.7826 -2.2035
-0.5379 -0.4596
-1.3799  0.0037
-0.3815  1.4219
 0.2192  0.2297
 1.0010  0.5570
 0.6981 -0.8828
 0.7232 -1.2942
 0.8049 -0.5391
 0.3457 -1.5950
-2.1742  2.4071
 0.9592  1.3948
-0.8991 -0.3394
-1.2080 -1.3720
-0.8710 -0.3277
 0.3054 -0.9281
-0.4665 -0.8242
-0.5735 -1.1918
-1.2887 -1.7625
[ CPUFloatType{20,2} ]


In [6]:
std::cout << torch::ones({X.size(0),1});

 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[ CPUFloatType{20,1} ]

In [7]:
torch::Tensor X_with_one = torch::cat({torch::ones({X.size(0),1}), X},1);

In [8]:
std::cout << X_with_one << std::endl;

 1.0000  1.5724 -2.3880
 1.0000  0.7826 -2.2035
 1.0000 -0.5379 -0.4596
 1.0000 -1.3799  0.0037
 1.0000 -0.3815  1.4219
 1.0000  0.2192  0.2297
 1.0000  1.0010  0.5570
 1.0000  0.6981 -0.8828
 1.0000  0.7232 -1.2942
 1.0000  0.8049 -0.5391
 1.0000  0.3457 -1.5950
 1.0000 -2.1742  2.4071
 1.0000  0.9592  1.3948
 1.0000 -0.8991 -0.3394
 1.0000 -1.2080 -1.3720
 1.0000 -0.8710 -0.3277
 1.0000  0.3054 -0.9281
 1.0000 -0.4665 -0.8242
 1.0000 -0.5735 -1.1918
 1.0000 -1.2887 -1.7625
[ CPUFloatType{20,3} ]


In [9]:
//torch::Tensor a = torch::randn({3,3});
//std::cout << torch::linalg::det(a);

## step2 use torch::linalg::lstsq to get regression coefficients

In [10]:
std::tuple<torch::Tensor,torch::Tensor,torch::Tensor,torch::Tensor> output = torch::linalg::lstsq(X_with_one,y, c10::nullopt, c10::nullopt);

In [11]:
torch::Tensor intercepts_and_coefficients = std::get<0>(output);
std::cout << intercepts_and_coefficients << std::endl;

 4.1983
 1.9977
-3.4002
[ CPUFloatType{3,1} ]


In [12]:
torch::Tensor y_pred_ts = X_with_one.matmul(intercepts_and_coefficients);

In [13]:
std::cout << y_pred_ts << std::endl;

 15.4591
 13.2539
  4.6864
  1.4293
 -1.3984
  3.8552
  4.3042
  8.5946
 10.0436
  7.6392
 10.3123
 -8.3296
  1.3720
  3.5563
  6.4502
  3.5727
  7.9643
  6.0689
  7.1050
  7.6169
[ CPUFloatType{20,1} ]


In [14]:
torch::Tensor residuals = y - y_pred_ts;
std::cout << residuals << std::endl;

0.01 *
-0.1383
  0.9323
 -1.0848
  0.1140
  0.7324
 -1.1629
  0.4797
  1.1288
 -1.9850
  0.7236
  0.2756
  0.3702
 -0.1542
  0.3493
  0.9991
 -1.5726
  0.5464
 -0.6239
 -0.2475
  0.3183
[ CPUFloatType{20,1} ]


## 2 batch data regression

In [17]:
torch::Tensor batch_X_with_one = torch::stack({X_with_one, X_with_one});

In [19]:
std::cout << batch_X_with_one << std::endl;
std::cout << batch_X_with_one.sizes() << std::endl;

(1,.,.) = 
  1.0000  1.5724 -2.3880
  1.0000  0.7826 -2.2035
  1.0000 -0.5379 -0.4596
  1.0000 -1.3799  0.0037
  1.0000 -0.3815  1.4219
  1.0000  0.2192  0.2297
  1.0000  1.0010  0.5570
  1.0000  0.6981 -0.8828
  1.0000  0.7232 -1.2942
  1.0000  0.8049 -0.5391
  1.0000  0.3457 -1.5950
  1.0000 -2.1742  2.4071
  1.0000  0.9592  1.3948
  1.0000 -0.8991 -0.3394
  1.0000 -1.2080 -1.3720
  1.0000 -0.8710 -0.3277
  1.0000  0.3054 -0.9281
  1.0000 -0.4665 -0.8242
  1.0000 -0.5735 -1.1918
  1.0000 -1.2887 -1.7625

(2,.,.) = 
  1.0000  1.5724 -2.3880
  1.0000  0.7826 -2.2035
  1.0000 -0.5379 -0.4596
  1.0000 -1.3799  0.0037
  1.0000 -0.3815  1.4219
  1.0000  0.2192  0.2297
  1.0000  1.0010  0.5570
  1.0000  0.6981 -0.8828
  1.0000  0.7232 -1.2942
  1.0000  0.8049 -0.5391
  1.0000  0.3457 -1.5950
  1.0000 -2.1742  2.4071
  1.0000  0.9592  1.3948
  1.0000 -0.8991 -0.3394
  1.0000 -1.2080 -1.3720
  1.0000 -0.8710 -0.3277
  1.0000  0.3054 -0.9281
  1.0000 -0.4665 -0.8242
  1.0000 -0.5735 -1.1918
  

In [20]:
torch::Tensor batch_y = torch::stack({y, y});

In [21]:
std::cout << batch_y << std::endl;
std::cout << batch_y.sizes() << std::endl;

(1,.,.) = 
  15.4577
  13.2633
   4.6756
   1.4304
  -1.3911
   3.8435
   4.3090
   8.6059
  10.0237
   7.6464
  10.3151
  -8.3259
   1.3704
   3.5598
   6.4602
   3.5569
   7.9698
   6.0626
   7.1026
   7.6201

(2,.,.) = 
  15.4577
  13.2633
   4.6756
   1.4304
  -1.3911
   3.8435
   4.3090
   8.6059
  10.0237
   7.6464
  10.3151
  -8.3259
   1.3704
   3.5598
   6.4602
   3.5569
   7.9698
   6.0626
   7.1026
   7.6201
[ CPUFloatType{2,20,1} ]
[2, 20, 1]


In [22]:
std::tuple<torch::Tensor,torch::Tensor,torch::Tensor,torch::Tensor> batch_output = torch::linalg::lstsq(batch_X_with_one,batch_y, c10::nullopt, c10::nullopt);

In [23]:
torch::Tensor batch_intercepts_and_coefficients = std::get<0>(batch_output);
std::cout << batch_intercepts_and_coefficients << std::endl;

(1,.,.) = 
  4.1983
  1.9977
 -3.4002

(2,.,.) = 
  4.1983
  1.9977
 -3.4002
[ CPUFloatType{2,3,1} ]


## appendix: understand c10::optional

In [15]:
c10::optional<double> a = c10::nullopt;
std::cout << a.has_value() << std::endl;

c10::optional<double> b(-1.1);
std::cout << b.has_value() << std::endl;;
std::cout << b.value() << std::endl;

0
1
-1.1
